# Card Scoring Validation Notebook

**Purpose:** Validate the new CardScorer implementation against Phase 1.5 data and known staple cards.

**Objectives:**
1. Test scoring engine with 3 known commanders
2. Verify known staples appear in top recommendations
3. Validate component scores make intuitive sense
4. Benchmark performance (must score 73K cards in <30s)
5. Document results and any necessary weight adjustments

**Created:** January 23, 2026

## Section 1: Import Libraries and Load Data

In [1]:
import sys
import os
import time
import pandas as pd
import numpy as np
import logging
from typing import Dict, List, Tuple, Any
from collections import defaultdict

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Add workspace to path
sys.path.insert(0, '/workspaces/mtgecorec')

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
# Load Phase 1.5 data files
DATA_PATH = '/workspaces/mtgecorec/notebooks/'

print("Loading Phase 1.5 data files...")

# Main card database
all_cards = pd.read_csv(os.path.join(DATA_PATH, 'master_analysis_full.csv'))
print(f"✓ Loaded {len(all_cards)} cards from master_analysis_full.csv")

# Mechanic synergy weights
mechanic_weights = pd.read_csv(os.path.join(DATA_PATH, 'mechanic_synergy_weights.csv'))
print(f"✓ Loaded {len(mechanic_weights)} mechanic weights")

# Archetype-mechanic alignment
archetype_weights = pd.read_csv(os.path.join(DATA_PATH, 'archetype_mechanic_weights.csv'))
print(f"✓ Loaded {len(archetype_weights)} archetype-mechanic mappings")

# Combo card database
combo_cards = pd.read_csv(os.path.join(DATA_PATH, 'combo_cards_list.csv'))
print(f"✓ Loaded {len(combo_cards)} combo cards")

# Mechanic co-occurrence matrix
cooccurrence_matrix = pd.read_csv(os.path.join(DATA_PATH, 'mechanic_cooccurrence_matrix.csv'), index_col=0)
print(f"✓ Loaded mechanic co-occurrence matrix")

print("\nData loading complete!")

Loading Phase 1.5 data files...
✓ Loaded 73063 cards from master_analysis_full.csv
✓ Loaded 50 mechanic weights
✓ Loaded 700 archetype-mechanic mappings
✓ Loaded 12061 combo cards
✓ Loaded mechanic co-occurrence matrix

Data loading complete!


## Section 2: Verify Test Commanders Exist

In [4]:
# Find our test commanders in the database
test_commanders = [
    'Meren of Clan Nel Toth',
    'Atraxa, Praetors\' Voice',
    'The Gitrog Monster'
]

# Also check for variations
test_variations = [
    'Meren of Clan Nel Toth',
    'Atraxa Praetors\' Voice',  # Without comma
    'Atraxa, Praetors\' Voice',
    'The Gitrog Monster',
    'Gitrog Monster'
]

print("Searching for test commanders...\n")

found_commanders = {}
for variation in test_variations:
    matches = all_cards[all_cards['name'].str.lower() == variation.lower()]
    if len(matches) > 0:
        commander = matches.iloc[0]
        found_commanders[variation] = commander.to_dict()
        print(f"✓ Found: {variation}")
        print(f"  Rarity: {commander.get('rarity')}, CMC: {commander.get('cmc')}, Mechanics: {commander.get('mechanic_count')}\n")
    else:
        print(f"✗ Not found: {variation}")

# Store the ones we found
meren = found_commanders.get('Meren of Clan Nel Toth')
atraxa = found_commanders.get('Atraxa, Praetors\' Voice') or found_commanders.get('Atraxa Praetors\' Voice')
gitrog = found_commanders.get('The Gitrog Monster') or found_commanders.get('Gitrog Monster')

if meren and atraxa and gitrog:
    print("\n✓ All test commanders found!")
else:
    print("\n⚠ Some commanders not found. Checking all cards...")
    print(all_cards[all_cards['name'].str.contains('Meren|Atraxa|Gitrog', case=False)][['name', 'rarity']].head(10))

Searching for test commanders...

✓ Found: Meren of Clan Nel Toth
  Rarity: mythic, CMC: 4.0, Mechanics: 1

✗ Not found: Atraxa Praetors' Voice
✓ Found: Atraxa, Praetors' Voice
  Rarity: mythic, CMC: 4.0, Mechanics: 6

✓ Found: The Gitrog Monster
  Rarity: mythic, CMC: 5.0, Mechanics: 3

✗ Not found: Gitrog Monster

✓ All test commanders found!


## Section 3: Import CardScorer from Implementation

In [26]:
# Import the CardScorer we just created
try:
    from core.data_engine.card_scoring import CardScorer
    print("✓ Successfully imported CardScorer")
except ImportError as e:
    print(f"✗ Failed to import CardScorer: {e}")
    print("\nFalling back to inline implementation...")

✓ Successfully imported CardScorer


## Section 4: Test Scoring on Meren of Clan Nel Toth

In [27]:
# Initialize scorer
print("Initializing CardScorer...")
scorer = CardScorer(data_path=DATA_PATH)
print("✓ CardScorer initialized\n")

# Find Meren
meren_row = all_cards[all_cards['name'] == 'Meren of Clan Nel Toth']
if len(meren_row) > 0:
    meren = meren_row.iloc[0].to_dict()
    print(f"Testing with: {meren['name']}")
    print(f"  Rarity: {meren.get('rarity')}")
    print(f"  CMC: {meren.get('cmc')}")
    print(f"  Mechanics: {meren.get('mechanic_count')}\n")
    
    # Score all cards for Meren
    print("Scoring all cards for Meren...")
    start_time = time.time()
    meren_scores = scorer.score_all_cards(meren)
    elapsed = time.time() - start_time
    
    print(f"✓ Scored {len(meren_scores)} cards in {elapsed:.2f} seconds")
    print(f"  Performance: {len(meren_scores)/elapsed:.0f} cards/second\n")
else:
    print("✗ Meren not found in database")

INFO:core.data_engine.card_scoring:Initializing CardScorer from /workspaces/mtgecorec/notebooks/
INFO:core.data_engine.card_scoring:Loaded 73063 cards from master_analysis_full.csv
INFO:core.data_engine.card_scoring:Loaded 50 mechanic weights
INFO:core.data_engine.card_scoring:Loaded 700 archetype-mechanic mappings
INFO:core.data_engine.card_scoring:Loaded 12061 combo cards
INFO:core.data_engine.card_scoring:Loaded mechanic co-occurrence matrix


Initializing CardScorer...


INFO:core.data_engine.card_scoring:Lookup tables built
INFO:core.data_engine.card_scoring:CardScorer initialized successfully
INFO:core.data_engine.card_scoring:Scoring all cards for Meren of Clan Nel Toth


✓ CardScorer initialized

Testing with: Meren of Clan Nel Toth
  Rarity: mythic
  CMC: 4.0
  Mechanics: 1

Scoring all cards for Meren...


INFO:core.data_engine.card_scoring:Scoring complete: 73063 cards scored


✓ Scored 73063 cards in 3.64 seconds
  Performance: 20048 cards/second



## Section 5: Display Top Recommendations for Meren

In [28]:
# Display top 20 recommendations for Meren
top_20_meren = meren_scores.head(20)

print("\n=== TOP 20 RECOMMENDATIONS FOR MEREN ===")
print("\nRank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type")
print("-" * 95)

for idx, (_, row) in enumerate(top_20_meren.iterrows(), 1):
    print(f"{idx:2d}    {row['card_name']:28s} {row['total_score']:5.1f}  {row['base_power']:5.1f}  {row['mechanic_synergy']:6.1f}  {row['archetype_fit']:7.1f}    {row['combo_bonus']:5.1f}  {row['curve_fit']:5.1f}  {row['type_balance']:5.1f}")

print("\n" + "="*95)

# Known staples for Meren
known_staples = ['Sakura-Tribe Elder', 'Eternal Witness', 'Spore Frog', 
                  'Viscera Seer', 'Blood Artist', 'Zulaport Cutthroat']

print("\n✓ STAPLE CARD VERIFICATION:")
for staple in known_staples:
    staple_data = meren_scores[meren_scores['card_name'] == staple]
    if len(staple_data) > 0:
        rank = list(meren_scores.index).index(staple_data.index[0]) + 1
        score = staple_data.iloc[0]['total_score']
        print(f"  {staple:30s} → Rank #{rank:3d} with score {score:.1f}/100")
    else:
        print(f"  {staple:30s} → NOT FOUND in database")


=== TOP 20 RECOMMENDATIONS FOR MEREN ===

Rank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type
-----------------------------------------------------------------------------------------------
 1    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 2    Sam, Loyal Attendant          75.7   66.0   100.0     65.0     30.0  100.0  100.0
 3    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 4    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 5    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 6    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 7    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 8    Silvar, Devourer of the Free  75.7   66.0   100.0     65.0     30.0  100.0  100.0
 9    Sam, Loyal Attendant          75.7   66.0   100.0     65.0     3

## Section 6: Detailed Component Breakdown for Top Cards

In [20]:
# Show detailed component breakdown for top 5 cards
print("\n=== DETAILED COMPONENT BREAKDOWN - MEREN TOP 5 ===")

for idx, (_, row) in enumerate(meren_scores.head(5).iterrows(), 1):
    print(f"\n{idx}. {row['card_name']}")
    print(f"   Total Score:       {row['total_score']:6.1f}/100")
    print(f"   ├─ Base Power:     {row['base_power']:6.1f}/100 (card inherent quality)")
    print(f"   ├─ Mechanic Synergi: {row['mechanic_synergy']:6.1f}/100 (30% weight - overlap with commander)")
    print(f"   ├─ Archetype Fit:  {row['archetype_fit']:6.1f}/100 (25% weight - strategy alignment)")
    print(f"   ├─ Combo Bonus:    {row['combo_bonus']:6.1f}/100 (15% weight - combo potential)")
    print(f"   ├─ Curve Fit:      {row['curve_fit']:6.1f}/100 (10% weight - mana curve balance)")
    print(f"   ├─ Type Balance:   {row['type_balance']:6.1f}/100 (5% weight - type distribution)")
    print(f"   └─ Color Multiplier: {row['color_multiplier']:6.2f}x (commander color identity)")


=== DETAILED COMPONENT BREAKDOWN - MEREN TOP 5 ===

1. Taigam, Master Opportunist
   Total Score:         75.7/100
   ├─ Base Power:       66.0/100 (card inherent quality)
   ├─ Mechanic Synergi:  100.0/100 (30% weight - overlap with commander)
   ├─ Archetype Fit:    65.0/100 (25% weight - strategy alignment)
   ├─ Combo Bonus:      30.0/100 (15% weight - combo potential)
   ├─ Curve Fit:       100.0/100 (10% weight - mana curve balance)
   ├─ Type Balance:    100.0/100 (5% weight - type distribution)
   └─ Color Multiplier:   1.00x (commander color identity)

2. Sam, Loyal Attendant
   Total Score:         75.7/100
   ├─ Base Power:       66.0/100 (card inherent quality)
   ├─ Mechanic Synergi:  100.0/100 (30% weight - overlap with commander)
   ├─ Archetype Fit:    65.0/100 (25% weight - strategy alignment)
   ├─ Combo Bonus:      30.0/100 (15% weight - combo potential)
   ├─ Curve Fit:       100.0/100 (10% weight - mana curve balance)
   ├─ Type Balance:    100.0/100 (5% weight - 

## Section 7: Test Scoring on Atraxa, Praetors' Voice

In [21]:
# Find and score Atraxa
atraxa_matches = all_cards[all_cards['name'].str.contains('Atraxa', case=False)]

if len(atraxa_matches) > 0:
    atraxa = atraxa_matches.iloc[0].to_dict()
    print(f"Testing with: {atraxa['name']}")
    print(f"  Rarity: {atraxa.get('rarity')}")
    print(f"  CMC: {atraxa.get('cmc')}")
    print(f"  Mechanics: {atraxa.get('mechanic_count')}\n")
    
    # Score all cards for Atraxa
    print("Scoring all cards for Atraxa...")
    start_time = time.time()
    atraxa_scores = scorer.score_all_cards(atraxa)
    elapsed = time.time() - start_time
    
    print(f"✓ Scored {len(atraxa_scores)} cards in {elapsed:.2f} seconds\n")
    
    # Display top 20
    print("\n=== TOP 20 RECOMMENDATIONS FOR ATRAXA ===")
    print("\nRank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type")
    print("-" * 95)
    
    for idx, (_, row) in enumerate(atraxa_scores.head(20).iterrows(), 1):
        print(f"{idx:2d}    {row['card_name']:28s} {row['total_score']:5.1f}  {row['base_power']:5.1f}  {row['mechanic_synergy']:6.1f}  {row['archetype_fit']:7.1f}    {row['combo_bonus']:5.1f}  {row['curve_fit']:5.1f}  {row['type_balance']:5.1f}")
    
    # Verify Atraxa staples
    atraxa_staples = ['Doubling Season', 'Proliferate', 'Deepglow Skate', 'Contagion Engine']
    
    print("\n✓ STAPLE CARD VERIFICATION (Atraxa):")
    for staple in atraxa_staples:
        staple_data = atraxa_scores[atraxa_scores['card_name'].str.contains(staple, case=False)]
        if len(staple_data) > 0:
            rank = list(atraxa_scores.index).index(staple_data.index[0]) + 1
            score = staple_data.iloc[0]['total_score']
            found_name = staple_data.iloc[0]['card_name']
            print(f"  {found_name:30s} → Rank #{rank:3d} with score {score:.1f}/100")
        else:
            print(f"  {staple:30s} → NOT FOUND in database")
else:
    print("✗ Atraxa not found in database")

INFO:core.data_engine.card_scoring:Scoring all cards for Atraxa, Praetors' Voice


Testing with: Atraxa, Praetors' Voice
  Rarity: mythic
  CMC: 4.0
  Mechanics: 6

Scoring all cards for Atraxa...


INFO:core.data_engine.card_scoring:Scoring complete: 73063 cards scored


✓ Scored 73063 cards in 3.60 seconds


=== TOP 20 RECOMMENDATIONS FOR ATRAXA ===

Rank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type
-----------------------------------------------------------------------------------------------
 1    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 2    Sam, Loyal Attendant          75.7   66.0   100.0     65.0     30.0  100.0  100.0
 3    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 4    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 5    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 6    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 7    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 8    Silvar, Devourer of the Free  75.7   66.0   100.0     65.0     30.0  100.0  100.0
 9    Sam, Loyal Attendant     

## Section 8: Test Scoring on The Gitrog Monster

In [22]:
# Find and score Gitrog
gitrog_matches = all_cards[all_cards['name'].str.contains('Gitrog', case=False)]

if len(gitrog_matches) > 0:
    gitrog = gitrog_matches.iloc[0].to_dict()
    print(f"Testing with: {gitrog['name']}")
    print(f"  Rarity: {gitrog.get('rarity')}")
    print(f"  CMC: {gitrog.get('cmc')}")
    print(f"  Mechanics: {gitrog.get('mechanic_count')}\n")
    
    # Score all cards for Gitrog
    print("Scoring all cards for Gitrog...")
    start_time = time.time()
    gitrog_scores = scorer.score_all_cards(gitrog)
    elapsed = time.time() - start_time
    
    print(f"✓ Scored {len(gitrog_scores)} cards in {elapsed:.2f} seconds\n")
    
    # Display top 20
    print("\n=== TOP 20 RECOMMENDATIONS FOR GITROG ===")
    print("\nRank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type")
    print("-" * 95)
    
    for idx, (_, row) in enumerate(gitrog_scores.head(20).iterrows(), 1):
        print(f"{idx:2d}    {row['card_name']:28s} {row['total_score']:5.1f}  {row['base_power']:5.1f}  {row['mechanic_synergy']:6.1f}  {row['archetype_fit']:7.1f}    {row['combo_bonus']:5.1f}  {row['curve_fit']:5.1f}  {row['type_balance']:5.1f}")
    
    # Verify Gitrog staples
    gitrog_staples = ['Crucible of Worlds', 'Azusa', 'Life from the Loam', 'Exploration']
    
    print("\n✓ STAPLE CARD VERIFICATION (Gitrog):")
    for staple in gitrog_staples:
        staple_data = gitrog_scores[gitrog_scores['card_name'].str.contains(staple, case=False)]
        if len(staple_data) > 0:
            rank = list(gitrog_scores.index).index(staple_data.index[0]) + 1
            score = staple_data.iloc[0]['total_score']
            found_name = staple_data.iloc[0]['card_name']
            print(f"  {found_name:30s} → Rank #{rank:3d} with score {score:.1f}/100")
        else:
            print(f"  {staple:30s} → NOT FOUND in database")
else:
    print("✗ Gitrog not found in database")

INFO:core.data_engine.card_scoring:Scoring all cards for The Gitrog, Ravenous Ride


Testing with: The Gitrog, Ravenous Ride
  Rarity: mythic
  CMC: 5.0
  Mechanics: 4

Scoring all cards for Gitrog...


INFO:core.data_engine.card_scoring:Scoring complete: 73063 cards scored


✓ Scored 73063 cards in 3.65 seconds


=== TOP 20 RECOMMENDATIONS FOR GITROG ===

Rank  Card Name                    Total  Base  Synergy  Archetype  Combo  Curve  Type
-----------------------------------------------------------------------------------------------
 1    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 2    Sam, Loyal Attendant          75.7   66.0   100.0     65.0     30.0  100.0  100.0
 3    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 4    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 5    Taigam, Master Opportunist    75.7   66.0   100.0     65.0     30.0  100.0  100.0
 6    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 7    Rakdos, the Muscle            75.7   66.0   100.0     65.0     30.0  100.0  100.0
 8    Silvar, Devourer of the Free  75.7   66.0   100.0     65.0     30.0  100.0  100.0
 9    Sam, Loyal Attendant     

## Section 9: Performance Benchmarking

In [11]:
# Performance summary
print("\n" + "="*60)
print("PERFORMANCE BENCHMARKING SUMMARY")
print("="*60)

print(f"\nTotal cards scored: {len(all_cards):,}")
print(f"Scoring time (Meren): {elapsed:.2f} seconds")
print(f"Performance: {len(all_cards)/elapsed:.0f} cards/second")
print(f"\n✓ Performance Target: Score 73K cards in <30 seconds")
if elapsed < 30:
    print(f"✓ PASSED: {elapsed:.2f}s < 30s")
else:
    print(f"✗ FAILED: {elapsed:.2f}s >= 30s")

# Check for NaN/Inf values
print("\n" + "="*60)
print("DATA QUALITY CHECKS")
print("="*60)

nan_count = meren_scores.isna().sum().sum()
inf_count = np.isinf(meren_scores.select_dtypes(include=[np.number])).sum().sum()

print(f"\nNaN values: {nan_count}")
print(f"Inf values: {inf_count}")

if nan_count == 0 and inf_count == 0:
    print("✓ PASSED: No NaN or Inf values")
else:
    print(f"✗ FAILED: Found {nan_count} NaN and {inf_count} Inf values")

# Score distribution
print(f"\nScore Distribution:")
print(f"  Min:  {meren_scores['total_score'].min():.1f}")
print(f"  Max:  {meren_scores['total_score'].max():.1f}")
print(f"  Mean: {meren_scores['total_score'].mean():.1f}")
print(f"  Std:  {meren_scores['total_score'].std():.1f}")


PERFORMANCE BENCHMARKING SUMMARY

Total cards scored: 73,063
Scoring time (Meren): 3.74 seconds
Performance: 19513 cards/second

✓ Performance Target: Score 73K cards in <30 seconds
✓ PASSED: 3.74s < 30s

DATA QUALITY CHECKS

NaN values: 0
Inf values: 0
✓ PASSED: No NaN or Inf values

Score Distribution:
  Min:  36.5
  Max:  75.7
  Mean: 51.0
  Std:  6.2


## Section 10: Color Identity Validation

In [12]:
# Verify color identity is enforced (off-color cards should score 0)
print("\n" + "="*60)
print("COLOR IDENTITY ENFORCEMENT CHECK")
print("="*60)

# Meren is Golgari (BG), so red/white/blue cards should have color_multiplier = 0
meren_color_identity = 'BG'  # Black-Green

print(f"\nMeren color identity: {meren_color_identity}")
print("Expected: Red, White, Blue cards should have color_multiplier = 0")

# Find some off-color cards
off_color_examples = [
    'Lightning Bolt',  # Red
    'Counterspell',     # Blue
    'Wrath of God',     # White
    'Swords to Plowshares'  # White
]

print("\nVerifying off-color cards get multiplier = 0:")
for card_name in off_color_examples:
    card_data = meren_scores[meren_scores['card_name'].str.contains(card_name, case=False)]
    if len(card_data) > 0:
        multiplier = card_data.iloc[0]['color_multiplier']
        score = card_data.iloc[0]['total_score']
        found_name = card_data.iloc[0]['card_name']
        print(f"  {found_name:30s} → multiplier={multiplier:.2f}, score={score:.1f}")
    else:
        print(f"  {card_name:30s} → not in database")

# Check for on-color cards
on_color_examples = [
    'Swamp',
    'Forest',
    'Llanowar Elves'
]

print("\nVerifying on-color cards get multiplier = 1.0:")
for card_name in on_color_examples:
    card_data = meren_scores[meren_scores['card_name'].str.contains(card_name, case=False)]
    if len(card_data) > 0:
        multiplier = card_data.iloc[0]['color_multiplier']
        score = card_data.iloc[0]['total_score']
        found_name = card_data.iloc[0]['card_name']
        print(f"  {found_name:30s} → multiplier={multiplier:.2f}, score={score:.1f}")
    else:
        print(f"  {card_name:30s} → not in database")


COLOR IDENTITY ENFORCEMENT CHECK

Meren color identity: BG
Expected: Red, White, Blue cards should have color_multiplier = 0

Verifying off-color cards get multiplier = 0:
  Lightning Bolt                 → not in database
  Counterspell                   → multiplier=1.00, score=40.8
  Wrath of God                   → multiplier=1.00, score=44.4
  Swords to Plowshares           → multiplier=1.00, score=47.1

Verifying on-color cards get multiplier = 1.0:
  Sol'kanar the Swamp King       → multiplier=1.00, score=51.4
  Deep Forest Hermit             → multiplier=1.00, score=64.2
  Llanowar Elves                 → not in database


## Section 11: Summary and Validation Results

In [13]:
print("\n" + "="*70)
print(" "*15 + "VALIDATION RESULTS SUMMARY")
print("="*70)

# Check all success criteria
criteria = [
    ("Score 73K cards in <30s", elapsed < 30),
    ("Known staples in top 50", len(meren_scores[meren_scores['card_name'] == 'Sakura-Tribe Elder']) > 0),
    ("Off-color cards score 0", len(meren_scores[meren_scores['color_multiplier'] == 0]) > 0),
    ("Component scores 0-100", meren_scores[['base_power', 'mechanic_synergy', 'archetype_fit', 'combo_bonus', 'curve_fit', 'type_balance']].min().min() >= 0),
    ("No NaN values", nan_count == 0),
    ("No Inf values", inf_count == 0),
]

print("\nMUST HAVE CRITERIA:")
pass_count = 0
for criterion, result in criteria:
    status = "✓ PASS" if result else "✗ FAIL"
    print(f"  {status} - {criterion}")
    if result:
        pass_count += 1

print(f"\n{pass_count}/{len(criteria)} criteria passed")

print("\n" + "="*70)
print("COMPONENT WEIGHT SUMMARY")
print("="*70)

weights = {
    "Base Power": 0.15,
    "Mechanic Synergy": 0.30,
    "Archetype Fit": 0.25,
    "Combo Bonus": 0.15,
    "Curve Fit": 0.10,
    "Type Balance": 0.05
}

print("\nComponent Weights (as specified in SPEC):")
for component, weight in weights.items():
    print(f"  {component:20s} {weight*100:5.1f}%")

total_weight = sum(weights.values())
print(f"  {'TOTAL':20s} {total_weight*100:5.1f}%")

print("\n" + "="*70)
if pass_count == len(criteria):
    print("\n✓✓✓ VALIDATION PASSED - CardScorer is ready for production! ✓✓✓")
else:
    print(f"\n⚠⚠⚠ VALIDATION ISSUES - {len(criteria) - pass_count} criteria failed ⚠⚠⚠")

print("="*70)


               VALIDATION RESULTS SUMMARY

MUST HAVE CRITERIA:
  ✓ PASS - Score 73K cards in <30s
  ✓ PASS - Known staples in top 50
  ✗ FAIL - Off-color cards score 0
  ✓ PASS - Component scores 0-100
  ✓ PASS - No NaN values
  ✓ PASS - No Inf values

5/6 criteria passed

COMPONENT WEIGHT SUMMARY

Component Weights (as specified in SPEC):
  Base Power            15.0%
  Mechanic Synergy      30.0%
  Archetype Fit         25.0%
  Combo Bonus           15.0%
  Curve Fit             10.0%
  Type Balance           5.0%
  TOTAL                100.0%


⚠⚠⚠ VALIDATION ISSUES - 1 criteria failed ⚠⚠⚠


## Next Steps

✓ CardScorer implementation complete
✓ ScoringAdapter for backward compatibility created
✓ Feature flag added to commander_recommender.py
✓ Validation notebook executed

### Phase 2: Integration
1. Run parallel A/B testing (50% new, 50% old scoring)
2. Monitor for API contract violations
3. Validate frontend displays correctly
4. Compare outputs with EDHREC rankings (target >0.7 correlation)

### Phase 3: Production Rollout
1. Set `USE_NEW_SCORING=true` (default: false)
2. Monitor for 48 hours
3. If successful, remove legacy recommendation code
4. Document performance improvements

## CRITICAL BUGS IDENTIFIED

**Root Cause Analysis:**

The CardScorer implementation has **placeholder/stub methods** instead of real mechanic synergy calculation:

```python
# Current BROKEN code in card_scoring.py line ~320:
mechanic_synergy = min(card_mechanics_count * 10 + 30, 100)
```

This is why:
1. ❌ All commanders get identical top 20 (commander ignored!)
2. ❌ Synergy always 100 for cards with 7+ mechanics (formula maxes out)
3. ❌ Known staples rank terribly (simple cards = low mechanic_count = low score)
4. ❌ No actual mechanic overlap calculation (Phase 1.5 data unused!)

**Required Fix:** Implement proper mechanic extraction and overlap scoring using:
- Parse actual mechanics from oracle_text or cluster data
- Compare card mechanics with commander mechanics (Jaccard similarity)
- Use mechanic_synergy_weights.csv for weighted scoring
- Use mechanic_cooccurrence_matrix.csv for co-occurrence bonuses

**Status:** Need complete rewrite of scoring components (spec for Haiku)

In [ ]:
# Diagnostic: Verify the broken mechanic synergy calculation
print("="*70)
print("DIAGNOSTIC: Mechanic Synergy Calculation Bug")
print("="*70)

# Sample cards with different mechanic counts
test_cards = [
    ('Card with 1 mechanic', 1),
    ('Card with 3 mechanics', 3),
    ('Card with 5 mechanics', 5),
    ('Card with 7 mechanics', 7),
    ('Card with 10 mechanics', 10),
]

print("\nBROKEN FORMULA: mechanic_synergy = min(mechanic_count * 10 + 30, 100)")
print("\nExpected behavior: Score should depend on OVERLAP with commander")
print("Actual behavior: Score only depends on card's mechanic_count\n")

for card_name, mech_count in test_cards:
    # This is what the broken code does:
    broken_score = min(mech_count * 10 + 30, 100)
    print(f"  {card_name:30s} → {broken_score}/100")

print("\n✗ Result: Any card with 7+ mechanics always scores 100")
print("✗ Result: Commander is completely ignored")
print("✗ Result: Simple cards (1-2 mechanics) always score low (40-50)")
print("\n✓ FIX: Implement proper mechanic overlap (Jaccard similarity)")
print("✓ FIX: Use mechanic_synergy_weights.csv for weighted importance")
print("✓ FIX: Use mechanic_cooccurrence_matrix.csv for bonuses")
print("\n" + "="*70)